<font color='green' size=6> HBO Max TV Shows and Movies </font>

---

# 1 - Definição do Problema

Analisar os dados da base de dados da HBO Max, afim de gerar um modelo capaz de recomendar filmes para usuários que tenham o gosto parecido. Além de gerar análise de shows, popularidade e afins

# 2 - Coleta de Dados

In [ ]:
# Importando módulos inicialmente necessários
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Importando a base de dados para um objeto dataframe
credits = pd.read_csv('data/credits.csv')
titles = pd.read_csv('data/titles.csv')

In [ ]:
# Visualizando as 3 primeiras observações da base de dados "credits"
credits.head(3)

In [ ]:
# Visualizando as 3 primeiras observações da base de dados "titles"
titles.head(3)

# 3 - Limpeza e Preparação de Dados

## 3.1 - Dados Dos Atores

In [ ]:
# Dimensão da base de dados
credits.shape

In [ ]:
# Visualização mais abrangente da base de dados
credits.head(10)

In [ ]:
# Visualização do sumário da base de dados
credits.info()

In [ ]:
# Observando se há dados nulos
credits.isna().sum()

In [ ]:
# Excluindo dados nulos
credits.dropna(inplace=True)

## 3.2 - Limpeza de Dados dos Filmes

In [ ]:
# Visualizando a base de dados
titles.head(3)

In [ ]:
# Visualizando um sumário da base de dados
titles.describe()

In [ ]:
# Visualizando informações da base de dados
titles.info()

In [ ]:
# Proporção de dados nulos
titles.isna().sum() / len(titles)

In [ ]:
# Excluindo colunas que não serão usadas
titles.drop(columns=['seasons', 'description'], inplace=True)

In [ ]:
# Limpando a coluna de certificação de idade
titles['age_certification'] = titles['age_certification'].fillna('G')

In [ ]:
# Limpando o resto da base de dados
titles.fillna(0, inplace=True)

In [ ]:
# Visualizando todos os genêros incluidos
genres = set()
for lg in titles['genres']:
    genres.update(eval(lg))

In [ ]:
# Adicionando novas colunas a base de dados
for gen in genres:
    titles[gen] = titles['genres'].apply(lambda x : gen in eval(x))
titles.drop(columns=['genres'], inplace=True)

In [ ]:
# Visualizando todos os países incluidos
countries = set()
for lc in titles['production_countries']:
    countries.update(eval(lc))

In [ ]:
# Adicionando novas colunas a base de dados
for country in countries:
    titles[country] = titles['production_countries'].apply(lambda x : country in eval(x))
titles.drop(columns=['production_countries'], inplace=True)

In [ ]:
# Alterando valores booleanos para valores númerios
titles.replace({True: 1, False : 0}, inplace=True)

# 4 - Análise Exploratória de Dados

## 4.1 - Análise da Base de Créditos

In [ ]:
# Dimensão da base de dados
credits.shape

In [ ]:
# Separando dados segundo os nomes
names = pd.value_counts(credits['name'])

In [ ]:
# Escolhendo os 15 atores que mais apareceram
names = names.sort_values(ascending=False)[:15]

In [ ]:
# Visualizando os atores que mais apareceram nos filmes/shows da HBO Max
plt.style.use('ggplot')
axes = sns.barplot(x=names.values, y=names.index, palette='mako')
axes.figure.set_size_inches(10, 5)
axes.set_title('Total de aparições de atores na HBO Max', fontsize=14)
axes.set_xlabel('Contagem')

## 4.2 - Análise da Base de Títulos

In [ ]:
# Diferença de dados de filme de classificação de idade
_ = pd.value_counts(titles['age_certification'])
sns.barplot(x=_.index, y=_.values)

In [ ]:
# O total de votos na HBO por classificação de idade
_ = titles.groupby(by='age_certification')['imdb_votes'].mean()
_ = _.sort_values(ascending=False)
sns.barplot(x=_.index, y=_.values)

In [ ]:
# A média de nota bruta por classificação de idade
_ = titles.groupby(by='age_certification')['imdb_score'].mean()
_ = _.sort_values(ascending=False)
sns.barplot(x=_.index, y=_.values)

In [ ]:
movies = titles.query('type == "MOVIE"')
shows = titles.query('type == "SHOW"')

In [ ]:
# Os 10 programas melhores avaliados da HBO Max
titles.query('imdb_votes > 284').sort_values(by='imdb_score', ascending=False).iloc[:10, :10]

In [ ]:
# Os 10 filmes mais avaliados
movies.query('imdb_votes > 284').sort_values(by='imdb_score', ascending=False).iloc[:10, :10]

In [ ]:
# Os 10 shows mais avaliados
shows.query('imdb_votes > 284').sort_values(by='imdb_score', ascending=False).iloc[:10, :10]

In [ ]:
# Método inicial de recomendação de filmes por gênero
def recommend_by_genre(gender, number):
    recommend_titles = titles[titles[gender] == 1]
    recommend_titles = recommend_titles.query('imdb_votes > 248')
    recommend_titles = recommend_titles.sort_values(by='imdb_score', ascending=False)
    return recommend_titles.iloc[:number, :10]

In [ ]:
recommend_by_genre('fantasy', 10)

# 5 - Criação do Modelo

In [ ]:
# Definindo as variáveis binárias dos gêneros
genres = titles.iloc[:, 11:30]

In [ ]:
# Criação e treino do modelo
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=20, n_init=10)

kmeans.fit(genres)

In [ ]:
# Definindo a variável target na base de dados
genres['target'] = kmeans.labels_
titles['target'] = kmeans.labels_

In [ ]:
# Salvando o arquivo em formato csv
genres.to_csv('data/genres.csv', index=False)

# 6 - Deploy Do Modelo

In [ ]:
with open('models/hbomax_model.pickle', 'wb') as f:
    pickle.dump(kmeans, f)

In [ ]:
def recommend_titles(x, number):
    x = [int(g in x) for g in genres.columns[:-1]]
    target = kmeans.predict([x])[0]
    target_titles = titles[titles['target'] == target]
    target_titles = target_titles.query('imdb_votes > 248')
    return target_titles.sort_values(by='imdb_score', ascending=False).iloc[:number, :10]

In [ ]:
recommend_titles(['fantasy', 'drama'], 10)